In [1]:
from typing import (
    List,
    Dict
)
from itertools import product
from functools import partial
from pdb import set_trace

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import pairwise_kernels

import gc4eptn
from gc4eptn import dataloaders
from gc4eptn.utils import norms
from gc4eptn.pngs import (
    run_ggm_pngs_exp, 
    run_pngs, 
)
from gc4eptn.utils.utils import (
    get_dataloader,
    to_adjacency,
    build_experiment_path,
    filter_function_kwargs,
    lambda_grid_R_max_min,
    lambda_grid_R_max_square,
    exp_log_grid,
)
from gc4eptn.utils.plotting import plot_eptn_ground_truths, R_mag_ang_plots
from gc4eptn.utils.kernels import (
    weighted_kernel,
    corr,
    cov,
    gram,
    rational_quadratic,
)
from gc4eptn.utils.metrics import (
    graph_fscore,
    graph_fbscore,
    graph_precision,
    graph_recall,
    hamming_distance,
    ebic,
)
np.set_printoptions(suppress=True)
%load_ext autoreload
%autoreload 2

In [2]:
rqk = partial(rational_quadratic, length_scale=50, alpha=1.0)
rbf = partial(pairwise_kernels, metric='rbf')

wcorr = partial(weighted_kernel, kernel=corr)
wcov = partial(weighted_kernel, kernel=cov)
wgram = partial(weighted_kernel, kernel=partial(gram, transpose=True))
wrq = partial(weighted_kernel, kernel=rqk)
wrbf = partial(weighted_kernel, kernel=rbf)

In [3]:
fstandardize = partial(
    norms.feature_norm,
    norm_fn=norms.standardize,
    features=['V_mag', 'V_ang', 'I_ang', 'I_mag']
)

In [4]:
metrics = {
    'hd': dict(
        fn=hamming_distance,
        type='lowest'
    ),
    'ebic': dict(
        fn=partial(ebic, gamma=0.5), 
        type='lowest'
    ),
    'f-score': dict(
        fn=graph_fscore,
        type='highest'
    ),
    'fb-score': dict(
        fn=partial(graph_fbscore, beta=2),
        type='highest'
    ),
    'precision': dict(
        fn=graph_precision,
        type='highest'
    ),
    'recall': dict(
        fn=graph_recall,
        type='highest'
    ),
}

def compute_metrics(A, A_hat, directed=False):
    results = {}
    results['f-score'] = metrics['f-score']['fn'](A, A_hat)
    results['fb-score'] = metrics['fb-score']['fn'](A, A_hat)
    results['precision'] = metrics['precision']['fn'](A, A_hat)
    results['recall'] = metrics['recall']['fn'](A, A_hat)
    results['hd'] = metrics['hd']['fn'](A, A_hat, directed)
    return results

lambda_grid=exp_log_grid(start=0.001, end=200, n=100, round_dec=4)
ggmncv_kwargs = {
    'penalize_diagonal': False,
    'LLA': False,
    'select': 'lambda',
    'lambda': lambda_grid, 
    'ic': 'ebic'
}

In [11]:
def run_pngs_exps(
    pngs_score_type = 'sum',
    pngs_min_threshold=0.2,
    metrics: Dict = metrics,
    best_metric_names: List[str] = ['fb-score', 'ebic'], 
    n_samples: int = [1800],
    difficulties: List[str] = ['novice'],
    topologies = ['complete', 'partial-left', 'partial-right'],
    load_types: List[str] = ['high', 'medium', 'low'],
    net_pmu_kwargs: List[Dict] = [None],
    norm_types: List[str] = ['fstandardize'],
    R_func_names: List[str] = ['rqk'],
    penalties: List[str] = ['lasso', 'scad', 'mcp', 'atan', 'selo'],
    ggm_kwargs: Dict = ggmncv_kwargs,
    seed: int = 0,
    verbose: bool = False,
):  
    """ Automates running of multiple GGM experiements for the RTDS v5
        This is an automated version of the pngs-single-exp notebook. 
    
        1st loop determines the pmu related arguments such as
        difficulty, topology, load, and net pmu kwargs.

        2nd loop determines the normalization method, number of samples,
        and kernel matrix type.

        3rd loop determines by the types of metrics to be used for selecting
        best graph.

        4th loop (inner moost) detrermines the type of GGM algorithm (penalty type) to be ran.
    """
    pmu = list(product(difficulties, topologies, load_types, net_pmu_kwargs))
    build = list(product(norm_types, n_samples, R_func_names))
    ########################################################
    for difficulty, topology, load, net_kwargs in tqdm(pmu):
        pmuds_class = get_dataloader('rtds', 'v5', difficulty)

        net_kwargs = net_kwargs if net_kwargs is not None else {}
        net_kwargs['load'] = load
        net_kwargs['topology'] = topology
        
        flow_pmuds = pmuds_class(
            load=load,
            topology=topology,
            drop_current=False,
            drop_parallel_currents=False,
        )
        flow_pmuds.file_name = 'Data_pu_Complete.csv'
        flow_pmuds.load_data()
        
        net_pmuds = pmuds_class(**net_kwargs).load_data()

        data_type_path = build_experiment_path(
            subdirs=[
                'rtds',
                'v5',
                'pngs',
                'multi-exp',
                difficulty,
                topology,
                'flow-multi-edge',
                '-'.join(load) if isinstance(load, list) else load,
            ]
        ) 
        print(f"Data type path: {data_type_path}")
        plot_eptn_ground_truths(
            data_type_path, 
            flow_pmuds=flow_pmuds,
            net_pmuds=net_pmuds,
            annot_kws=dict(fontsize=12))
        #######################################
        for norm_type, n, R_func_name in build:
            norm_fn = globals()[norm_type] if  norm_type is not None else None
            R_func = globals()[R_func_name]

            net_pmuds.build_graph_data(
                n=n, 
                norm_fn=norm_fn,
                random=True,
                rng=np.random.default_rng(seed=seed),
             )
            R = R_func(net_pmuds.graph_df)
            
            save_dir_path = build_experiment_path(
                subdirs=[
                    norm_type,
                    n if n is not None else len(net_pmuds.df),
                    R_func_name,
                ],
                path=data_type_path
            )
            print(f"Save dir path: {save_dir_path}")
  
            plt.clf()
            plt.close('all')
            R_mag_ang_plots(
                R_func=R_func,
                pmuds_class=pmuds_class,
                load_kwargs=dict(
                    load=load,
                    topology=topology, 
                ),
                build_kwargs=dict(
                    n=n,
                    random=True,
                    norm_fn=norm_fn,
                    rng=np.random.default_rng(seed=seed),
                ),
                filename=R_func_name,
                annot_kws=dict(fontsize=12),
                save_path=save_dir_path
            )

            ################################################
            for select_best_metric_name in best_metric_names:
                select_best_metric = metrics[select_best_metric_name]
                save_path_score = save_dir_path/select_best_metric_name
                algo_metrics_s1 = {}
                algo_metrics_s2 = {}
                ggm_params = {}

                ########################
                for penalty in penalties:
                    s1_metrics, s2_metrics, params, pngs = run_ggm_pngs_exp(
                        net_pmuds=net_pmuds,
                        flow_pmuds=flow_pmuds,
                        R=R,
                        penalty=penalty,
                        n_samples=n,
                        pngs_score_type=pngs_score_type,
                        pngs_min_threshold=pngs_min_threshold,
                        select_best_metric_fn=select_best_metric['fn'],
                        select_best_metric_type=select_best_metric['type'],
                        ggm_metrics_fn=compute_metrics,
                        refined_metrics_fn=partial(compute_metrics, directed=True),
                        ggmncv_kwargs=ggmncv_kwargs,
                        save_path=save_path_score,
                        include_ggm_result_metrics=['ic'],
                        verbose=verbose,
                        disable_plotting=True,
                    )
                    algo_metrics_s1[penalty] = s1_metrics
                    algo_metrics_s2[penalty] = s2_metrics
                    ggm_params[penalty] = params
                    # prevent plots from being stored past saving
                    plt.clf()
                    plt.close('all')
                    
                s1 = pd.DataFrame.from_dict(algo_metrics_s1, orient='index')
                s1.rename(columns=dict(score=select_best_metric_name), inplace=True)
                s1.index = s1.index.str.upper()
                s1.to_csv(save_path_score/'stage-1-metrics.csv')
                with open(save_path_score/'stage-1-metrics.tex', 'w') as tf:
                     tf.write(s1.to_latex(float_format="%.2f", longtable=False))

                s2 = pd.DataFrame.from_dict(algo_metrics_s2, orient='index')
                s2.index = s2.index.str.upper()
                s2.to_csv(save_path_score/'stage-2-metrics.csv')
                with open(save_path_score/'stage-2-metrics.tex', 'w') as tf:
                     tf.write(s2.to_latex(float_format="%.2f", longtable=False))
                
                params = pd.DataFrame.from_dict(ggm_params, orient='index')
                params.index = params.index.str.upper()
                params.to_csv(save_path_score/'ggm-params.csv')
                with open(save_path_score/'ggm-params.tex', 'w') as tf:
                     tf.write(params.to_latex(float_format="%.2f", longtable=False))
                del s1, s2, params
    plt.clf()
    plt.close('all')


Runs all experiments for every topology. For each topology, the high, medium, and low loads are tested. The GMM algorithms 'lasso', 'scad', 'mcp', 'atan', and 'selo' are used for each load.  Additionally, for each GGM algorithm the best graphs are chosen using 'fb-score', 'ebic', 'recall', 'f-score' metrics. 

In [33]:
run_pngs_exps(
    best_metric_names=['fb-score', 'ebic', 'recall', 'f-score'],
    net_pmu_kwargs=[dict(drop_parallel_currents=True)]
)

  0%|          | 0/9 [00:00<?, ?it/s]

Data type path: /home/ben/space/modl/gc4eptn/exps/v5/pngs-all/novice/complete/multi-edge
Save dir path: /home/ben/space/modl/gc4eptn/exps/v5/pngs-all/novice/complete/multi-edge/high/fstandardize/1800/rqk


R[write to console]: selecting lambda



  |======================================================================| 100%

R[write to console]: selecting lambda



...

  |======================================================================| 100%

R[write to console]: selecting lambda



  |======================================================================| 100%

R[write to console]: selecting lambda



  |======================================================================| 100%

Runs all experiments for every topology. For each topology, only the "all loads" type is tested. The GMM algorithms 'lasso', 'scad', 'mcp', 'atan', and 'selo' are used for each load.  Additionally, for each GGM algorithm the best graphs are chosen using 'fb-score', 'ebic', 'recall', 'f-score' metrics. 

In [12]:
run_pngs_exps(
    load_types=[['high', 'medium', 'low']],
    n_samples=[5400],
    best_metric_names=['fb-score', 'ebic', 'recall', 'f-score'],
    net_pmu_kwargs=[dict(drop_parallel_currents=True)]
)


  0%|          | 0/3 [00:00<?, ?it/s]

Data type path: /home/ben/space/modl/gc4eptn/exps/rtds/v5/pngs/multi-exp/novice/complete/multi-edge/high-medium-low
Save dir path: /home/ben/space/modl/gc4eptn/exps/rtds/v5/pngs/multi-exp/novice/complete/multi-edge/high-medium-low/fstandardize/5400/rqk


R[write to console]: selecting lambda



  |======================================================================| 100%

R[write to console]: selecting lambda



...

  |======================================================================| 100%

R[write to console]: selecting lambda



  |======================================================================| 100%

R[write to console]: selecting lambda



  |======================================================================| 100%